In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
df = pd.read_csv(r'C:\Code\Analisis\analisis_harga_bitcoin_2025\dataset_btc_pro.csv', parse_dates=["Date"])
df = df.sort_values("Date").reset_index(drop=True)

# pastikan numerik
num_cols = ["Close","Pembukaan","Tertinggi","Terendah","Perubahan","persentase"]
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")

df.dropna(inplace=True)

features_lstm = ["Close","Pembukaan","Tertinggi","Terendah"]

data = df[features_lstm].values
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

X, y = [], []
window = 14

In [ ]:
for i in range(window, len(data_scaled)):
    X.append(data_scaled[i-window:i])
    y.append(data_scaled[i, 0])  # target Close

X, y = np.array(X), np.array(y)

split = int(len(X)*0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [ ]:
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.2),
    LSTM(25),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse")
model.fit(X_train, y_train, epochs=30, batch_size=16, validation_data=(X_test,y_test))